In [1]:
from __future__ import print_function
from IPython.display import display
import datetime, calendar, nltk, string, math, csv, glob, ntpath, os.path
import numpy as np
from gensim.models import Phrases
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer; wnl = WordNetLemmatizer()
import warnings, matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
#from rirdc_lib_old import cleanUp, calculate_seed
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()
from sklearn.cluster import KMeans
import matplotlib.image as mpimg
import pandas as pd

#Give the wordnet path here
wordnet_path=r'C:\\nltk_data\\corpora\\wn-domains-3.2\\wn-domains-3.2-20070223'
path = r'C:\Users\shilp\Documents' 
stopwords_file, categories_file = path+'\\stopwords.txt', path+'\\Category_DomainTerms.txt' # {(category, term)}
data_file=path+'\\results-20170725-145833.csv'
out_fname= path+'\\new_data_file.csv'
extra_cats_file=path+'\\extra_cats.txt'
n_topics, seed_ratio = 20, 0.25
max_df, min_df = 0.5, 10 # For the VSM
topic_groups=[]
#from rirdc_lib_old import cleanUp, calculate_seed
def keywithmaxval(d):
    v=list(d.values());k=list(d.keys())
    return k[v.index(max(v))]

C:\Users\shilp\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Creates a new file from the tweet file by getting rid of rows that are incomplete by appending their content to the 
#preceding row and deleting blank rows
test_file=open(data_file,encoding="ISO-8859-1")
twitter_file = open(out_fname, 'w',encoding="ISO-8859-1",newline='')
reader = csv.reader(test_file, delimiter=',')
writer = csv.writer(twitter_file)
next(reader,None)
prev=reader
for row in reader:
    if row[0] in (None, ""):
        continue
    for term in row[3]:
        if term.isalpha()==False:
            try:
                if math.isnan(float(term)):
                    print("Entered")
                    row[3]=row[3].replace(term,' ')
            except:
                    pass
        if ord(term) > 127:
            row[3]=row[3].replace(term,' ')
        wnl.lemmatize(term,'n')
    writer.writerow(row)
test_file.close()
twitter_file.close()
print('Done Creating the cleaned version of tweet file...')

Done Creating the cleaned version of tweet file...


In [3]:
# Loading the Wordnet domains.
s2d = []
for i in open(wordnet_path, 'r'):
    ssid, doms = i.strip().split('\t')
    doms = doms.split()
    for d in doms:
        s2d.append(d)
synset2domains=list(set(s2d))
extra_cats={}

for syn in synset2domains:
    try:
        p_list,c_list,categories=[],[],[]
        obj=wn.synset(syn+'.n.01')
        topics=obj.topic_domains()
        parents=obj.hypernyms()
        children=obj.hyponyms()
        domains= [ topic.lemma_names() for topic in topics ] 
        for sublist in domains:
            for item in sublist:
                    categories.append(str(item.replace('_',' ')))
        pwords = [ parent.lemma_names() for parent in parents ]
        for sublist in pwords:
            for item in sublist:
                    p_list.append(str(item.replace('_',' ')))
        cwords = [ child.lemma_names() for child in children]
        for sublist in cwords:
            for item in sublist:
                    c_list.append(str(item.replace('_',' ')))
        words=p_list+c_list
        if categories:
            for category in categories:
                extra_cats[category]=words
    except:
        pass

out_extra_cats = open(extra_cats_file, 'w')
for k in extra_cats.keys():
    for v in extra_cats[k]:
        out_extra_cats.write(k+", ")
        out_extra_cats.write(v+"\n")
out_extra_cats.close()
print("Done creating extra categories..")

Done creating extra categories..


In [4]:
input_file = open(out_fname,encoding="ISO-8859-1")
date_list,country_list=[],[]
reader = csv.reader(input_file, delimiter=',')

#Creates a unique list of months and countries from tweets
for row in reader:
    created_at,*rest=row[1].split(' ')           #splits the 'Created at' column to retrieve 'month'
    year,months,date=created_at.split('-')
    month=calendar.month_name[int(months)]
    if month not in date_list:
       date_list.append(str(month))
    if row[6] not in country_list:
       country_list.append(str(row[6]))

#Creates csv files with names after months
for month in date_list:
   outfile = open(path+'\Data{0:2s}'.format(str(month))+'.csv', 'w', newline='',encoding="ISO-8859-1")
   writer = csv.writer(outfile, delimiter = ',')
   writer.writerow(["tweet_id","month","text","country"])
   outfile.close()
 
#Creates csv files with names after countries
for country in country_list:
   outfile = open(path+'\Data{0:2s}'.format(str(country))+'.csv', 'w', newline='',encoding="ISO-8859-1")
   writer = csv.writer(outfile, delimiter = ',')
   writer.writerow(["tweet_id","month","text","country"])
   outfile.close()
    
#Shifts rows of tweets to their respective csv files  -->  For Months
for month1 in date_list:
  with open(out_fname, 'r',encoding="ISO-8859-1") as f:
    reader = csv.reader(f, delimiter=',')
    outfile = open(path+'\Data{0:2s}'.format(str(month1))+'.csv','a', newline='',encoding="ISO-8859-1")
    for row in reader:
        if row[0] in (None, ""):
          continue
        created_at,*rest=row[1].split(' ')  
        year,months,date=created_at.split('-')
        month2=calendar.month_name[int(months)]
        if month1==month2:
            writer = csv.writer(outfile, delimiter =',')
            writer.writerow(row)
    outfile.close()
 
#Shifts rows of tweets to their respective csv files  -->  For Countries
for country1 in country_list:
  with open(out_fname, 'r',encoding="ISO-8859-1") as f:
    reader = csv.reader(f, delimiter=',')
    outfile = open(path+'\Data{0:2s}'.format(str(country1))+'.csv','a', newline='',encoding="ISO-8859-1")
    for row in reader:
        if row[0] in (None, ""):
          continue
        country2=str(row[6])
        if country1==country2:
            writer = csv.writer(outfile, delimiter =',')
            writer.writerow(row)
    outfile.close()
print('Done moving tweets to their respective months and location files...')

Done moving tweets to their respective months and location files...


In [5]:
#Tokenizes and remove stopwords from the tweet column of the data file for bigrams and unigrams --> For Months

df=open(stopwords_file,"r",encoding="ISO-8859-1", errors='replace')
stops=df.readlines(); df.close()
stops = set([word.strip() for word in stops])                  #stores stopwords tokens from the stoplist in a list
stops1=list(stops)
stops2=''.join(stops1)
tweet_id_m, text_m={},{}
tweet_id_c,text_c={},{}
docs,r=[],[]
sentences=[]
DocZ_m, DocZ_c, DocZ_m_wob, DocZ_c_wob = {},{},{},{} # will be used to find categories
for month in date_list:
    u,p,d,d_wob=[],[],[],[]
    outfile = open(r'C:\Users\shilp\Documents\Data{0:2s}'.format(str(month))+'.csv', 'r', newline='',encoding="ISO-8859-1")
    reader = csv.reader(outfile, delimiter=',')
    next(reader,None)
    docs=list(reader);
    for doc in docs:    
        tmp = [t.strip() for t in doc]
        try:
            if len(tmp[3])>0: # Making sure not blanks
                sentence = [word
                    for word in nltk.word_tokenize(tmp[3].lower())
                    if word not in string.punctuation
                           and len(word)>3
                           ]
                filtered_word_list = sentence[:] #make a copy of the word_list
                for term in sentence: # iterate over word_list
                    if term in stops2: 
                        filtered_word_list.remove(term)
                d_wob.append(tmp[3].lower())                  #Unigrams List
                sentences.append(filtered_word_list)                  #Bigrams List
                filtered=' '.join(filtered_word_list)
                u.append(tmp[0]); p.append(tmp[3]);d.append(filtered)
        except:
            pass
    tweet_id_m[month]=u
    text_m[month]=p 
    DocZ_m[month]=d
    DocZ_m_wob[month]=d_wob
   
print("For the months found")
#print("The bigrams are -->", DocZ_m)
#print("The unigrams are -->", DocZ_m_wob)

#Gets the top five bigrams
flat_list = [item for sublist in sentences for item in sublist]
word_counter = {}
for word in flat_list:
     if word in word_counter:
        word_counter[word] += 1
     else:
         word_counter[word] = 1
popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
top_5 = popular_words[:5]
print("The top five bigrams are",top_5)
del u,p,d,d_wob,sentence
    
del docs
print("Clean-up for months done...,", end = ' ', flush = True)

For the months found
The top five bigrams are ['just', 'love', 'today', 'good', 'people']
Clean-up for months done..., 

In [6]:
'''
#Tokenizes and remove stopwords from the tweet column of the data file for bigrams and unigrams --> For Countries
for country in country_list:
    u,p,d,d_wob=[],[],[],[]
    outfile = open(r'C:\Users\shilp\Documents\Data{0:2s}'.format(str(country))+'.csv', 'r', newline='',encoding="ISO-8859-1")
    reader = csv.reader(outfile, delimiter=',')
    next(reader,None)
    docs=list(reader);
    for doc in docs:    
       tmp = [t.strip() for t in doc]
       try:
         if len(tmp[3])>7: # Making sure not blanks
            sentence = [word
                    for word in nltk.word_tokenize(tmp[3].lower())
                    if word not in string.punctuation
                        and len(word)>3]
            filtered_word_list = sentence[:] #make a copy of the word_list
            for term in sentence: # iterate over word_list
                    if term in stops2: 
                        filtered_word_list.remove(term)
            d_wob.append(tmp[3].lower())
            filtered=' '.join(filtered_word_list)
            u.append(tmp[0]); p.append(tmp[3]);d.append(filtered)
       except:
           pass
    tweet_id_c[country]=u
    text_c[country]=p  #make phrase
    DocZ_c[country]=d
    DocZ_c_wob[country]=d_wob
    
print("For the countries found")
print("The bigrams are -->", DocZ_c)
print("The unigrams are -->", DocZ_c_wob)
    
print("Clean-up for countries done...,", end = ' ', flush = True)
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 195-196: truncated \UXXXXXXXX escape (<ipython-input-6-2e4a3d555084>, line 36)

In [7]:
# Loading categories into category dictionary of set of terms for bigrams and unigrams for months

print("Please wait (it's going to take a while): Loading Data,", end = ' ', flush = True)
df=open(categories_file,"r",encoding="ISO-8859-1", errors='replace')
cats=df.readlines(); df.close(); categories = {}
for cat in cats:
     key = cat.split(', ')[0].strip(); term = cat.split(', ')[1].strip().lower()
     try:        
          categories[key].add(term)
     except:
          categories[key] = set([term])               #stores categories-terms as a key-value in a dictionary from the file
del cats,df

df=open(extra_cats_file,"r",encoding="ISO-8859-1", errors='replace')
cats=df.readlines(); df.close(); 
for cat in cats:
     key = cat.split(', ')[0].strip(); term = cat.split(', ')[1].strip().lower()
     try:        
          categories[key].add(term)
     except:
          categories[key] = set([term])   
print("pre-Categories mapping ... ", flush = True)    
doc_cats_m,doc_cat_m = {}, {}; del cats, df
doc_cats_m_wob,doc_cat_m_wob={},{};
for month in date_list:
   doc_cats_m[month],doc_cat_m[month]={},{} 
   for idx, doc in enumerate(DocZ_m[month]):
       notInCategories,total=[],0
       doc_cats_m[month][idx]={}
       for cat, terms in categories.items():
          for term in terms:
             n = doc.count(term)              #calculates the frequency of occurance of terms corresponding to terms for the category
             total+=n
             try:
                doc_cats_m[month][idx][cat] += n
             except:
                doc_cats_m[month][idx][cat]= n
       for cat, terms in categories.items():
            if total==0:
               doc_cat_m[month][idx] = None
               doc_cats_m[month][idx][cat] = 0.0
               if month not in notInCategories:
                  notInCategories.append(month)
            else:
             doc_cat_m[month][idx] = keywithmaxval(doc_cats_m[month][idx])           # category of doc[idx]
             doc_cats_m[month][idx][cat] = doc_cats_m[month][idx][cat]/total          # Normalizing to category's ratio
                
del doc
#print(doc_cat_m)
print('%d/%d Documents were not in any of the categories.' %(len(notInCategories), len(DocZ_m)), flush = True)

#For unigrams
for month in date_list:
   doc_cats_m_wob[month],doc_cat_m_wob[month]={},{} 
   for idx, doc in enumerate(DocZ_m_wob[month]):
       notInCategories,total=[],0
       doc_cats_m_wob[month][idx]={}
       for cat, terms in categories.items():
          for term in terms:
             n = doc.count(term)
             total+=n
             try:
                doc_cats_m_wob[month][idx][cat] += n
             except:
                doc_cats_m_wob[month][idx][cat]= n
       for cat, terms in categories.items():
            if total==0:
               doc_cat_m_wob[month][idx] = None
               doc_cats_m_wob[month][idx][cat] = 0.0
               if month not in notInCategories:
                  notInCategories.append(month)
            else:
             doc_cat_m_wob[month][idx] = keywithmaxval(doc_cats_m_wob[month][idx])# category of doc[month][idx]
             doc_cats_m_wob[month][idx][cat] = doc_cats_m_wob[month][idx][cat]/total # Normalizing to category's ratio

print("------------------------------------------------------------------------------------------------------------------------------------------>>>>>")
#print(doc_cat_m_wob)
print('%d/%d Documents were not in any of the categories.' %(len(notInCategories), len(DocZ_m_wob)), flush = True)
print('Done mapping categories...', flush = True)

Please wait (it's going to take a while): Loading Data, pre-Categories mapping ... 
0/4 Documents were not in any of the categories.
------------------------------------------------------------------------------------------------------------------------------------------>>>>>
0/4 Documents were not in any of the categories.
Done mapping categories...


In [8]:
# Loading categories into category dictionary of set of terms for bigrams and unigrams for countries
'''
print("Please wait (it's going to take a while): Loading Data,", end = ' ', flush = True)
doc_cats_c,doc_cat_c = {}, {};
doc_cats_c_wob,doc_cat_c_wob={},{};
for country in country_list:
   doc_cats_c[country],doc_cat_c[country]={},{} 
   for idx, doc in enumerate(DocZ_c[country]):
       notInCategories,total=[],0
       doc_cats_c[country][idx]={}
       for cat, terms in categories.items():
          for term in terms:
             n = doc.count(term)
             total+=n
             try:
                doc_cats_c[country][idx][cat] += n
             except:
                doc_cats_c[country][idx][cat]= n
       for cat, terms in categories.items():
            if total==0:
               doc_cat_c[country][idx] = None
               doc_cats_c[country][idx][cat] = 0.0
               if country not in notInCategories:
                  notInCategories.append(country)
            else:
             doc_cat_c[country][idx] = keywithmaxval(doc_cats_c[country][idx])# category of doc[idx]
             doc_cats_c[country][idx][cat] = doc_cats_c[country][idx][cat]/total # Normalizing to category's ratio
                
#print(doc_cat_c)
#print('%d/%d Documents were not in any of the categories.' %(len(notInCategories), len(DocZ_c)), flush = True)

for country in country_list:
   doc_cats_c_wob[country],doc_cat_c_wob[country]={},{} 
   for idx, doc in enumerate(DocZ_c_wob[country]):
       notInCategories,total=[],0
       doc_cats_c_wob[country][idx]={}
       for cat, terms in categories.items():
          for term in terms:
             n = doc.count(term)
             total+=n
             try:
                doc_cats_c_wob[country][idx][cat] += n
             except:
                doc_cats_c_wob[country][idx][cat]= n
       for cat, terms in categories.items():
            if total==0:
               doc_cat_c_wob[country][idx] = None
               doc_cats_c_wob[country][idx][cat] = 0.0
               if month not in notInCategories:
                  notInCategories.append(country)
            else:
             doc_cat_c_wob[country][idx] = keywithmaxval(doc_cats_c_wob[country][idx])# category of doc[idx]
             doc_cats_c_wob[country][idx][cat] = doc_cats_c_wob[country][idx][cat]/total # Normalizing to category's ratio

#print("------------------------------------------------------------------------------------------------------------------------------------------>>>>>")
#print(doc_cat_c_wob)
#print('%d/%d Documents were not in any of the categories.' %(len(notInCategories), len(DocZ_c_wob)), flush = True)
'''

'\nprint("Please wait (it\'s going to take a while): Loading Data,", end = \' \', flush = True)\ndoc_cats_c,doc_cat_c = {}, {};\ndoc_cats_c_wob,doc_cat_c_wob={},{};\nfor country in country_list:\n   doc_cats_c[country],doc_cat_c[country]={},{} \n   for idx, doc in enumerate(DocZ_c[country]):\n       notInCategories,total=[],0\n       doc_cats_c[country][idx]={}\n       for cat, terms in categories.items():\n          for term in terms:\n             n = doc.count(term)\n             total+=n\n             try:\n                doc_cats_c[country][idx][cat] += n\n             except:\n                doc_cats_c[country][idx][cat]= n\n       for cat, terms in categories.items():\n            if total==0:\n               doc_cat_c[country][idx] = None\n               doc_cats_c[country][idx][cat] = 0.0\n               if country not in notInCategories:\n                  notInCategories.append(country)\n            else:\n             doc_cat_c[country][idx] = keywithmaxval(doc_cats_c[cou

In [9]:
'''
#Prints out the documents that moved their categories between bigrams and unigrams for months
for month in date_list:
 print("For month",month)
 #shared_items = set(doc_cat[month].items()) & set(doc_cat_wob[month].items())
 results = [(k, doc_cat_m[month][k], doc_cat_m_wob[month][k]) for k in doc_cat_m[month] if k in doc_cat_m_wob[month] and doc_cat_m[month][k]!=doc_cat_m_wob[month][k]]
 print ("These are the differences in the categories that changed from with bigram to without bigram")
 print (results)
 print("---------------------->")
 doc_numbers=[result[0] for result in results]
 outfile = open(r'C:\Users\shilp\Documents\Data{0:2s}'.format(str(month))+'.csv', 'r', newline='',encoding="ISO-8859-1")
 reader = csv.reader(outfile, delimiter=',')
 next(reader,None)
 rows=list(reader); 
 print ("These are the documents that changed")
 #for number in doc_numbers:
  #          print("Document number", number); print(rows[number][3])
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 612-613: truncated \UXXXXXXXX escape (<ipython-input-9-a5f2716e3eb7>, line 18)

In [ ]:
'''
#Prints out the documents that moved their categories between bigrams and unigrams for countries
for country in country_list:
 print("For country",country)
 #shared_items = set(doc_cat[month].items()) & set(doc_cat_wob[month].items())
 results = [(k, doc_cat_c[country][k], doc_cat_c_wob[country][k]) for k in doc_cat_c[country] if k in doc_cat_c_wob[country] and doc_cat_c[country][k]!=doc_cat_c_wob[country][k]]
 #print (shared_items)
 print ("These are the differences in the categories that changed from with bigram to without bigram")
 #print (results)
 print("---------------------->")
 doc_numbers=[result[0] for result in results]
 outfile = open(r'C:\Users\shilp\Documents\Data{0:2s}'.format(str(country))+'.csv', 'r', newline='',encoding="ISO-8859-1")
 reader = csv.reader(outfile, delimiter=',')
 next(reader,None)
 rows=list(reader); 
 print ("These are the documents that changed")
 #for number in doc_numbers:
  #          print("Document number", number); print(rows[number][3])

In [10]:
#Creates tf-idf vocab list
print('Please wait calculating VSM ...', end = ' ', flush = True)
dtm_tf_m,dtm_tf_c={},{}
tf_terms_m,tf_terms_c={},{}
tf_vectorizer_m, tf_vectorizer_c={},{}

#For months
for m in date_list:
   tf_vectorizer_m[m] = CountVectorizer(strip_accents = 'unicode',stop_words = 'english',lowercase = True,token_pattern = r'\b[a-zA-Z]{3,}\b',max_df = 0.5,min_df=10)
   dtm_tf_m[m]=tf_vectorizer_m[m].fit_transform(DocZ_m[m])   #a sparse matrix
   #print("stm_tf is");print(dtm_tf[c])
   tf_terms_m[m] = tf_vectorizer_m[m].get_feature_names()      #a list
'''
#For countries
for c in country_list:
   tf_vectorizer_c[c] = CountVectorizer(strip_accents = 'unicode',stop_words = 'english',lowercase = True,token_pattern = r'\b[a-zA-Z]{3,}\b',max_df = 0.5, min_df=10)
   dtm_tf_c[c]=tf_vectorizer_c[c].fit_transform(DocZ_c[c])
   #print("stm_tf is");print(dtm_tf[c])
   tf_terms_c[c] = tf_vectorizer_c[c].get_feature_names()
'''

Please wait calculating VSM ... 

'\n#For countries\nfor c in country_list:\n   tf_vectorizer_c[c] = CountVectorizer(strip_accents = \'unicode\',stop_words = \'english\',lowercase = True,token_pattern = r\'\x08[a-zA-Z]{3,}\x08\',max_df = 0.5, min_df=10)\n   dtm_tf_c[c]=tf_vectorizer_c[c].fit_transform(DocZ_c[c])\n   #print("stm_tf is");print(dtm_tf[c])\n   tf_terms_c[c] = tf_vectorizer_c[c].get_feature_names()\n'

In [11]:
print('Please wait calculating LDA topics ...', end = ' ', flush = True)
with warnings.catch_warnings():
    lda_tf_m,vsm_topics_m,doc_topic_m={},{},{}
    lda_tf_c,vsm_topics_c,doc_topic_c={},{},{}
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    for m in date_list:
       lda= LatentDirichletAllocation(n_topics=10, max_iter=11,learning_method='batch', learning_offset=50., random_state=0)
       lda_tf_m[m]=lda.fit(dtm_tf_m[m])
       vsm_topics_m[m]=lda_tf_m[m].transform(dtm_tf_m[m])
       doc_topic_m[m]=[a.argmax()+1 for a in vsm_topics_m[m]] # topic of docs
    print(doc_topic_m)
'''  
    for c in country_list:
       lda= LatentDirichletAllocation(n_topics=10, max_iter=11,learning_method='batch', learning_offset=50., random_state=0)
       lda_tf_c[c]=lda.fit(dtm_tf_c[c])
       vsm_topics_c[c]=lda_tf_c[c].transform(dtm_tf_c[c])
       doc_topic_c[c]=[a.argmax()+1 for a in vsm_topics_c[c]]
'''
print('Done Assigning Topics', flush = True)

Please wait calculating LDA topics ... {'October': [1, 1, 1, 1, 1, 6, 6, 1, 1, 4, 1, 1, 3, 4, 1, 1, 1, 4, 9, 1, 1, 10, 6, 7, 8, 2, 7, 1, 1, 1, 1, 1, 5, 1, 1, 4, 1, 1, 1, 1, 1, 4, 6, 5, 1, 1, 4, 1, 1, 1, 1, 1, 5, 1, 10, 9, 5, 1, 7, 7, 3, 9, 1, 1, 7, 5, 7, 1, 6, 1, 10, 1, 1, 1, 1, 4, 7, 9, 9, 4, 10, 1, 8, 1, 5, 5, 1, 3, 5, 7, 1, 2, 5, 9, 8, 1, 1, 10, 6, 10, 1, 1, 6, 4, 1, 7, 2, 5, 2, 2, 1, 10, 1, 1, 7, 1, 1, 3, 3, 1, 1, 1, 1, 1, 2, 5, 1, 9, 1, 4, 1, 4, 6, 1, 1, 1, 7, 6, 1, 3, 8, 1, 1, 1, 7, 1, 3, 6, 1, 5, 6, 1, 1, 4, 1, 1, 1, 4, 1, 9, 1, 8, 10, 6, 5, 1, 4, 1, 1, 1, 6, 1, 8, 1, 9, 1, 2, 9, 8, 10, 7, 4, 1, 1, 9, 5, 5, 8, 1, 1, 2, 6, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 5, 6, 6, 3, 4, 8, 6, 3, 6, 4, 4, 4, 1, 5, 1, 2, 4, 1, 5, 3, 1, 4, 7, 1, 3, 5, 4, 7, 1, 4, 5, 6, 9, 10, 1, 8, 5, 4, 5, 1, 2, 4, 2, 1, 1, 2, 1, 4, 1, 1, 5, 10, 8, 1, 1, 6, 6, 1, 1, 1, 6, 1, 2, 1, 1, 5, 2, 2, 1, 1, 1, 7, 1, 6, 1, 6, 4, 6, 1, 6, 8, 6, 1, 10, 4, 1, 1, 1, 4, 5, 1, 7, 10, 1, 6, 1, 4, 5, 1, 4, 4, 7, 7, 1, 1, 4, 9

In [12]:
print('Please wait visualising LDA topics for different months ...', end = ' ', flush = True)
for m in date_list:
  print("For month -> ",m)
  display(pyLDAvis.sklearn.prepare(lda_tf_m[m], dtm_tf_m[m], tf_vectorizer_m[m]))

Please wait visualising LDA topics for different months ... For month ->  July


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      10.884331        1       1  0.001384 -0.204878
0      10.824871        1       2 -0.134637  0.088711
5      10.673836        1       3  0.017456 -0.276937
6      10.078982        1       4 -0.109017  0.157916
2      10.047212        1       5  0.103678 -0.136077
3       9.885673        1       6  0.214722  0.089120
4       9.555410        1       7  0.149771  0.144826
7       9.505602        1       8  0.168023  0.094948
1       9.482692        1       9 -0.201907  0.063429
9       9.061392        1      10 -0.209474 -0.021060, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
163   Default  173.000000           just  173.000000  30.0000  30.0000
184   Default  127.000000           love  127.000000  29.0000  29.0000
127   Default  113.000000           good  113.000000  28.0000  28.0000
216   Default  118.000000         people  118.000000  27.0000  27.0000
320   Default  125.000000          today  125.000000  26.0000  26.0000
129   Default   75.000000          great   75.000000  25.0000  25.0000
165   Default   68.000000           know   68.000000  24.0000  24.0000
204   Default   66.000000           need   66.000000  23.0000  23.0000
15    Default   58.000000      australia   58.000000  22.0000  22.0000
139   Default   56.000000          happy   56.000000  21.0000  21.0000
315   Default   56.000000          think   56.000000  20.0000  20.0000
337   Default   55.000000          video   55.000000  19.0000  19.0000
312   Default   54.000000          thank   54.000000  18.0000  18.0000
26    Default   54.000000           best   54.000000  17.0000  17.0000
354   Default   52.000000           work   52.000000  16.0000  16.0000
179   Default   48.000000           live   48.000000  15.0000  15.0000
356   Default   48.000000          world   48.000000  14.0000  14.0000
313   Default   51.000000         thanks   51.000000  13.0000  13.0000
181   Default   48.000000           look   48.000000  12.0000  12.0000
208   Default   60.000000          night   60.000000  11.0000  11.0000
28    Default   53.000000  billshortenmp   53.000000  10.0000  10.0000
364   Default   44.000000        youtube   44.000000   9.0000   9.0000
342   Default   69.000000           want   69.000000   8.0000   8.0000
13    Default   89.000000         auspol   89.000000   7.0000   7.0000
347   Default   50.000000        weekend   50.000000   6.0000   6.0000
346   Default   40.000000           week   40.000000   5.0000   5.0000
306   Default   51.000000         sydney   51.000000   4.0000   4.0000
254   Default   38.000000          right   38.000000   3.0000   3.0000
244   Default   40.000000         really   40.000000   2.0000   2.0000
293   Default   38.000000          start   38.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
215   Topic10   18.385058          party   19.244669   2.3555  -3.6381
132   Topic10   17.422500           haha   18.282099   2.3530  -3.6919
135   Topic10   16.459920           half   17.319529   2.3502  -3.7487
252   Topic10   15.497362        respect   16.356959   2.3472  -3.8090
104   Topic10   13.572233      followers   14.431819   2.3397  -3.9416
345   Topic10   12.609637       watching   13.469249   2.3352  -4.0152
264   Topic10   12.609637         saying   13.469248   2.3352  -4.0152
297   Topic10   12.609636         stream   13.469249   2.3352  -4.0152
174   Topic10   11.647084           link   12.506679   2.3299  -4.0946
198   Topic10   11.647075          month   12.506679   2.3299  -4.0946
339   Topic10   10.684527          vivid   11.544109   2.3238  -4.1808
213   Topic10   10.684525           para   11.544109   2.3238  -4.1808
96    Topic10   10.684505      favourite   11.544109   2.3238  -4.1809
155   Topic10    9.721945      instagram   

For month ->  August


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      11.247314        1       1  0.202433  0.007087
9      11.231766        1       2  0.235809 -0.026741
4      10.761112        1       3 -0.096087 -0.284014
0      10.072530        1       4 -0.024616  0.117183
5       9.928479        1       5 -0.227219  0.166300
3       9.874723        1       6 -0.077445 -0.176613
1       9.603952        1       7 -0.088111 -0.112094
6       9.337113        1       8  0.195301  0.087814
7       9.229074        1       9 -0.006853  0.024220
2       8.713937        1      10 -0.113211  0.196858, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
92    Default  116.000000            just  116.000000  30.0000  30.0000
68    Default   91.000000            good   91.000000  29.0000  29.0000
112   Default   75.000000            love   75.000000  28.0000  28.0000
5     Default   65.000000          auspol   65.000000  27.0000  27.0000
96    Default   61.000000            know   61.000000  26.0000  26.0000
134   Default   79.000000          people   79.000000  25.0000  25.0000
196   Default   53.000000           video   53.000000  24.0000  24.0000
70    Default   57.000000           great   57.000000  23.0000  23.0000
7     Default   50.000000       australia   50.000000  22.0000  22.0000
200   Default   57.000000            want   57.000000  21.0000  21.0000
185   Default   71.000000           today   71.000000  20.0000  20.0000
17    Default   44.000000            best   44.000000  19.0000  19.0000
76    Default   43.000000           happy   43.000000  18.0000  18.0000
125   Default   52.000000            need   52.000000  17.0000  17.0000
105   Default   40.000000            live   40.000000  16.0000  16.0000
136   Default   39.000000           photo   39.000000  15.0000  15.0000
188   Default   39.000000          travel   39.000000  14.0000  14.0000
181   Default   40.000000          thanks   40.000000  13.0000  13.0000
183   Default   37.000000           think   37.000000  12.0000  12.0000
148   Default   47.000000          really   47.000000  11.0000  11.0000
216   Default   33.000000         youtube   33.000000  10.0000  10.0000
61    Default   35.000000            game   35.000000   9.0000   9.0000
209   Default   34.000000           world   34.000000   8.0000   8.0000
180   Default   33.000000           thank   33.000000   7.0000   7.0000
151   Default   31.000000           right   31.000000   6.0000   6.0000
208   Default   34.000000            work   34.000000   5.0000   5.0000
18    Default   33.000000          better   33.000000   4.0000   4.0000
108   Default   39.000000            look   39.000000   3.0000   3.0000
66    Default   32.000000           going   32.000000   2.0000   2.0000
202   Default   31.000000            week   31.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
151   Topic10   31.131861           right   31.988574   2.4131  -2.5229
135   Topic10   18.118528          person   18.975243   2.3940  -3.0642
102   Topic10   17.117526           liked   17.974219   2.3914  -3.1210
33    Topic10   16.116489            care   16.973192   2.3885  -3.1812
86    Topic10   15.115457           house   15.972166   2.3851  -3.2454
0     Topic10   13.113413          abbott   13.970116   2.3770  -3.3875
210   Topic10   13.113412           wrong   13.970116   2.3770  -3.3875
67    Topic10   12.112395          gonski   12.969091   2.3719  -3.4669
206   Topic10   12.112372           woman   12.969090   2.3719  -3.4669
88    Topic10   10.110342         islands   10.967040   2.3589  -3.6475
37    Topic10   10.110311     choppergate   10.967040   2.3589  -3.6475
39    Topic10   12.081497            cool   14.924334   2.2289  -3.4694
93    Topic10   14.114578    justinbieber   18.927191   2.1469  -3.31

For month ->  September


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      12.310826        1       1 -0.234761  0.127878
0      10.887602        1       2 -0.243966 -0.013061
9      10.564418        1       3  0.094274  0.134646
5       9.722678        1       4 -0.067977 -0.177080
1       9.702923        1       5  0.265364  0.097042
6       9.560587        1       6  0.000116 -0.276592
2       9.544876        1       7  0.177402 -0.116272
4       9.328435        1       8 -0.020467  0.224125
7       9.274604        1       9  0.033176  0.043919
8       9.103051        1      10 -0.003160 -0.044605, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
156   Default  139.000000       love  139.000000  30.0000  30.0000
128   Default  175.000000       just  175.000000  29.0000  29.0000
99    Default  113.000000       good  113.000000  28.0000  28.0000
303   Default   80.000000       want   80.000000  27.0000  27.0000
134   Default   75.000000       know   75.000000  26.0000  26.0000
174   Default   69.000000       need   69.000000  25.0000  25.0000
282   Default  106.000000      today  106.000000  24.0000  24.0000
191   Default   69.000000      photo   69.000000  23.0000  23.0000
102   Default   68.000000      great   68.000000  22.0000  22.0000
275   Default   70.000000     thanks   70.000000  21.0000  21.0000
298   Default   59.000000      video   59.000000  20.0000  20.0000
144   Default   61.000000       life   61.000000  19.0000  19.0000
277   Default   57.000000      think   57.000000  18.0000  18.0000
169   Default   72.000000    morning   72.000000  17.0000  17.0000
222   Default   55.000000      right   55.000000  16.0000  16.0000
327   Default   48.000000    youtube   48.000000  15.0000  15.0000
16    Default   47.000000       best   47.000000  14.0000  14.0000
179   Default   49.000000      night   49.000000  13.0000  13.0000
318   Default   44.000000      world   44.000000  12.0000  12.0000
152   Default   39.000000       look   39.000000  11.0000  11.0000
88    Default   39.000000       game   39.000000  10.0000  10.0000
111   Default   39.000000      happy   39.000000   9.0000   9.0000
285   Default   37.000000    tonight   37.000000   8.0000   8.0000
274   Default   53.000000      thank   53.000000   7.0000   7.0000
151   Default   47.000000       live   47.000000   6.0000   6.0000
215   Default   48.000000     really   48.000000   5.0000   5.0000
324   Default   34.000000      years   34.000000   4.0000   4.0000
96    Default   44.000000      going   44.000000   3.0000   3.0000
75    Default   42.000000       feel   42.000000   2.0000   2.0000
136   Default   32.000000      labor   32.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
154   Topic10   21.380362      looks   22.240792   2.3571  -3.4013
261   Topic10   19.445477      story   20.305916   2.3533  -3.4962
70    Topic10   17.510610     family   18.371039   2.3486  -3.6010
197   Topic10   15.575729    playing   16.436163   2.3428  -3.7181
292   Topic10   14.608294       turn   15.468725   2.3393  -3.7822
147   Topic10   13.640864       link   14.501287   2.3354  -3.8507
108   Topic10   13.640850       hair   14.501287   2.3354  -3.8507
26    Topic10   12.673425    bronwyn   13.533849   2.3309  -3.9243
264   Topic10   11.705976    success   12.566411   2.3256  -4.0037
211   Topic10   10.738547     public   11.598973   2.3195  -4.0900
35    Topic10   10.738543  caulfield   11.598973   2.3195  -4.0900
33    Topic10   10.738533       case   11.598973   2.3195  -4.0900
216   Topic10   10.738532     reason   11.598973   2.3195  -4.0900
311   Topic10   10.738531     winter   11.598973   2.3195  -4.0900
20    Topic10   10.690560     bishop   11.600156   2.3149  -4.0944
234   Topic10    9.771124    seattle   10.631536   2.3122  -4.1844
207   Topic10    9.771115 

For month ->  October


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      12.570228        1       1 -0.305179 -0.133706
4      10.316679        1       2  0.054074 -0.022901
0      10.301106        1       3 -0.232103 -0.066553
2      10.003780        1       4  0.133987  0.117709
6       9.959201        1       5 -0.048207  0.143742
3       9.882585        1       6  0.074096 -0.238876
9       9.878147        1       7 -0.035529  0.231761
5       9.620828        1       8  0.198921 -0.039954
7       9.551142        1       9  0.173090 -0.160320
8       7.916305        1      10 -0.013150  0.169098, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
87    Default  123.000000              just  123.000000  30.0000  30.0000
174   Default   99.000000             today   99.000000  29.0000  29.0000
64    Default   81.000000              good   81.000000  28.0000  28.0000
118   Default   75.000000            people   75.000000  27.0000  27.0000
183   Default   78.000000             video   78.000000  26.0000  26.0000
100   Default   75.000000              love   75.000000  25.0000  25.0000
67    Default   64.000000             great   64.000000  24.0000  24.0000
13    Default   55.000000         beautiful   55.000000  23.0000  23.0000
168   Default   40.000000            thanks   40.000000  22.0000  22.0000
187   Default   44.000000              want   44.000000  21.0000  21.0000
170   Default   43.000000             think   43.000000  20.0000  20.0000
109   Default   42.000000           morning   42.000000  19.0000  19.0000
197   Default   36.000000              work   36.000000  18.0000  18.0000
63    Default   39.000000             going   39.000000  17.0000  17.0000
28    Default   44.000000     camerondallas   44.000000  16.0000  16.0000
111   Default   38.000000              need   38.000000  15.0000  15.0000
3     Default   42.000000  apositivemessage   42.000000  14.0000  14.0000
137   Default   36.000000             right   36.000000  13.0000  13.0000
6     Default   35.000000         australia   35.000000  12.0000  12.0000
132   Default   45.000000            really   45.000000  11.0000  11.0000
27    Default   40.000000           cameron   40.000000  10.0000  10.0000
91    Default   35.000000              left   35.000000   9.0000   9.0000
167   Default   39.000000          thankful   39.000000   8.0000   8.0000
71    Default   39.000000             happy   39.000000   7.0000   7.0000
162   Default   33.000000            sydney   33.000000   6.0000   6.0000
112   Default   29.000000              news   29.000000   5.0000   5.0000
89    Default   40.000000              know   40.000000   4.0000   4.0000
97    Default   35.000000              live   35.000000   3.0000   3.0000
93    Default   31.000000              life   31.000000   2.0000   2.0000
17    Default   31.000000            better   31.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
145   Topic10   16.347647            school   17.202030   2.4853  -3.0344
158   Topic10   13.301506             stuff   14.155880   2.4740  -3.2406
104   Topic10   13.301501             media   14.155880   2.4740  -3.2406
184   Topic10   12.286110              vote   13.140496   2.4690  -3.3200
149   Topic10   12.286104         seriously   13.140496   2.4690  -3.3200
201   Topic10   11.270755              yang   12.125114   2.4632  -3.4063
54    Topic10   11.270741           forward   12.125113   2.4632  -3.4063
147   Topic10   11.270731            second   12.125113   2.4632  -3.4063
88    Topic10   11.270729              kids   12.125113   2.4632  -3.4063
25    Topic10   10.255364        brooksbeau   11.109730   2.4562  -3.5007
115   Topic10   10.255346            online   11.109729   2.4562  -3.5007
74    Topic10   10.255345              head   11.1097

In [ ]:
'''
print('Please wait calculating LDA topics for different countries...', end = ' ', flush = True)
for c in country_list:
  print("For country -> ",c)
  display(pyLDAvis.sklearn.prepare(lda_tf_c[c], dtm_tf_c[c], tf_vectorizer_c[c]))
'''

'\nprint(\'Please wait calculating LDA topics for different countries...\', end = \' \', flush = True)\nfor c in country_list:\n  print("For country -> ",c)\n  display(pyLDAvis.sklearn.prepare(lda_tf_c[c], dtm_tf_c[c], tf_vectorizer_c[c]))\n'

In [ ]:
print("Assigning groups to topics...")
dict_topics_groups={}
for m in date_list:
    answer=[]
    print("Enter for month ",m)
    for top in range(0,5):
        value = [int(x) for x in input().split()]
        answer.append(value)
    dict_topics_groups[m]=answer
    print(dict_topics_groups)

In [ ]:
topic_categories_freq, topic_categories_score = {}, {}
qwerty=[]
for month in date_list:
#topics_groups = [[1,12],[2,18],[3,7,9,14,15,19],[4],[5],[6,8,11],[10,13,16,17],[20]] # Change this depending on the visualization result; PS: this is "LDAtopic+1"
#topics_scores, seeds, seeds_score, clusters, centroids, mse = {}, {}, {}, {}, {}, {}
 topic_categories_freq[month], topic_categories_score[month] = {}, {}
 for topic in dict_topics_groups[month]:
      temp=[]
      topic_categories_freq[month][str(topic)], topic_categories_score[month][str(topic)] = {}, {}
      doc_idx = [idx for idx, tpc in enumerate(doc_topic_m[month]) if tpc in topic] # all document index in the chosen topic    
      for idx in doc_idx:
        C = doc_cat_m[month][idx]
        temp.append(C)
        if C: #not None
            #print(C)
            try:
                topic_categories_freq[month][str(topic)][C]+=1
                topic_categories_score[month][str(topic)][C]+=doc_cats[month][idx][C]
            except:
                topic_categories_freq[month][str(topic)][C]=1
                topic_categories_score[month][str(topic)][C]=doc_cats[month][idx][C]
        else:
            try:
                topic_categories_freq[month][str(topic)]['None']+=1
            except:
                topic_categories_freq[month][str(topic)]['None']=1  
      for k,v in categories.items():
        if k not in temp:
           topic_categories_freq[month][str(topic)][k]=0.0
    # Normalizing scores
      for C in topic_categories_freq[month][str(topic)].keys():
        if C is not 'None':
            N = topic_categories_freq[month][str(topic)][C]
            if N !=0:
                topic_categories_score[month][str(topic)][C] = topic_categories_score[month][str(topic)][C]/N
            else:
                topic_categories_score[month][str(topic)][C] = 0.0 
print(topic_categories_freq)
#print(topic_categories_score)
    # doc_cats[idx][cat] ; 
    #seeds[str(topic)], seeds_score[str(topic)] = calculate_seed(dtm_tf,doc_topic,tf_terms,topic,categories,seed_ratio) # return Dictionary [categories]:{docs} ==> nested dict
    #clusters[str(topic)], centroids[str(topic)], mse[str(topic)] = ss_clustering(vsm_topics,seeds[str(topic)],doc_topic,topic) # k from len(seeds[str(topic)])

In [ ]:
#Calculates a score for each domain in each country
'''
topic_categories_freq_c, topic_categories_score_c = {}, {}
qwerty=[]
for country in country_list:
 topic_categories_freq_c[country], topic_categories_score_c[country] = {}, {}
 for topic in dict_topics_groups_c[country]:
      temp=[]
      topic_categories_freq_c[country][str(topic)], topic_categories_score_c[country][str(topic)] = {}, {}
      doc_idx = [idx for idx, tpc in enumerate(doc_topic_c[country]) if tpc in topic] # all document index in the chosen topic    
      for idx in doc_idx:
        C = doc_cat_c[country][idx]
        temp.append(C)
        if C: #not None
            try:
                topic_categories_freq_c[country][str(topic)][C]+=1
                topic_categories_score_c[country][str(topic)][C]+=doc_cats_c[country][idx][C]
            except:
                topic_categories_freq_c[country][str(topic)][C]=1
                topic_categories_score_c[country][str(topic)][C]=doc_cats_c[country][idx][C]
        else:
            try:
                topic_categories_freq_c[country][str(topic)]['None']+=1
            except:
                topic_categories_freq_c[country][str(topic)]['None']=1  
      for k,v in categories.items():
        if k not in temp:
            topic_categories_freq_c[country][str(topic)][k]=0.0
    # Normalizing scores
      for C in topic_categories_freq_c[country][str(topic)].keys():
        if C is not 'None':
            N = topic_categories_freq_c[country][str(topic)][C]
            if N !=0:
                topic_categories_score_c[country][str(topic)][C] = topic_categories_score_c[country][str(topic)][C]/N
            else:
                topic_categories_score_c[country][str(topic)][C] = 0.0
      if topic_categories_freq_c[country][str(topic)]['agriculture'] == topic_categories_freq_c[country][str(topic)]['finance'] == topic_categories_freq_c[country][str(topic)]['mining'] == 0.0:
        topic_categories_freq_c[country][str(topic)]['None']=1 
#print(topic_categories_freq_c)
#print(topic_categories_score_c)
print ('Done calculating score..')
'''

In [ ]:
#Creates a pie chart and saves it as a png file to be loaded later
fileNameTemplate = path+'\Plot{0:2s}'
categoryy_list=categories.keys()
categoryy_list=list(categoryy_list)
for month in date_list:
  print("For month",month)
  for itr,topic in enumerate(dict_topics_groups_m[month]): # e.g. 4 = [4,3,7,11]
#ss_visualize(topic,clusters[str(topic)],mse[str(topic)],seeds_score[str(topic)]) # Visualize the Semi-supervised categorization
     print('For Topic = ',topic)
     print('Visualization based on Category frequency: ', flush = True)
     y=np.array(list(topic_categories_freq[month][str(topic)].values()))
     percent = 100.*y/y.sum()
     indices=np.where(percent == 0)[0].tolist()
     porcent=np.delete(percent,indices)
     np.asarray(categoryy_list)
     category_list=np.delete(categoryy_list,indices).tolist()
     labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(category_list, porcent)]
     #labels = list(topic_categories_freq_c[country][str(topic)].keys())
     #print(labels)
     sizes = list(topic_categories_freq[month][str(topic)].values())
     colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','lightskyblue'][:len(labels)]
     explode = (0.1, 0, 0, 0)  # explode 1st slice: Change this anyway you like
     #plt.title
     patches,text=plt.pie(sizes, colors=colors, startangle=140)
     plt.axis('equal')
     sort_legend = True
     if sort_legend:
        patches, labels, dummy =  zip(*sorted(zip(patches, labels, sizes),
                                          key=lambda x: x[2],
                                          reverse=True))

     plt.legend(patches, labels, loc='left center', bbox_to_anchor=(-0.1, 1.),
           fontsize=8)

     plt.savefig(fileNameTemplate.format(str(topic)+'-'+month+'.png'), format='png', bbox_inches='tight')
     plt.show()
print('Done Visualising and saving pie charts...')

In [ ]:
#Loads back the saved png pie charts to be displayed over month-topic groups axes

n_clusters=5      #the one specified in kmm
fig, axarr = plt.subplots(nrows=len(date_list),ncols= n_clusters, sharex='col', sharey='row',  figsize=(40,5))
'''
file_list = glob.glob(path + '/*.png')
x = []
file_names=[]
y=[]
v=1
for file_path in file_list:
 head, tail = ntpath.split(file_path)
 x.append(tail.split('-',1)[0])
 file_names.append(tail.split('-',1)[1])
for file_name in file_names:
 y.append(file_name.split('.',1)[0])
print(y)
'''
#When there's a single month detected, use 1D array for co-ordinates
r=0
if len(date_list)==1:
    axarr[0].yaxis.set_label_position("left")
    axarr[0].set_ylabel(date_list[0])
    c=0
    for col in dict_topics_groups_m[date_list[0]]:
      ttl=''
      for top in col:
        ttl=ttl+mydict[str(top)]+','
      axarr[c].set_title(ttl)
      img=fileNameTemplate.format(str(col))+'-'+date_list[0]+'.png'
      o=axarr[c].imshow(mpimg.imread(img),interpolation='nearest', aspect='auto')
      plt.setp(axarr[c].get_xticklabels(), visible=False)
      plt.setp(axarr[c].get_yticklabels(), visible=False)
      c+=1
    
#else use numpy multi-dimesnional array
else:
 for row in date_list:
    c=0
    axarr[r,0].yaxis.set_label_position("left")
    axarr[r,0].set_ylabel(row)
    for col in dict_topics_groups_m[row]:  
      ttl=''
      for top in col:
        ttl=ttl+mydict[str(top)]+','
      #axarr[0,c].yaxis.set_label_position("top")
      axarr[r,c].set_title(ttl)
      img=fileNameTemplate.format(str(col))+'-'+row+'.png'
      #if os.path.isfile(img):
      #print("row");print(r);print("column"); print(c)
      o=axarr[r, c].imshow(mpimg.imread(img),interpolation='nearest',aspect='auto')
      plt.setp(axarr[r,c].get_xticklabels(), visible=False)
      plt.setp(axarr[r,c].get_yticklabels(), visible=False)
      c+=1
    r+=1
plt.show()